## Find UM radiation timesteps closest to the chosen phase angles

In [1]:
import numpy as np

In [2]:
from aeolus.const import init_const

In [3]:
from util_commons import PLANETS
from util_mypaths import path_to_constants

In [4]:
# Parameters
hatp11b_const = init_const("hatp11b", directory=path_to_constants)
hd189733b_const = init_const("hd189733b", directory=path_to_constants)
hd209458b_const = init_const("hd209458b", directory=path_to_constants)
wasp17b_const = init_const("wasp17b", directory=path_to_constants)
PLANET_CONSTS = [hatp11b_const, hd189733b_const, hd209458b_const, wasp17b_const]

In [5]:
def calc_star_planet_separation_angle(star_radius, planet_semi_major_axis):
    """Calculate the star-planet separation angle [degrees], which is the angle between (a) and (b), where
    (a) secondary eclipse, i.e., when a star eclipses a planet and star-planet centers are aligned, and
    (b) angle, when a maximum area of the planetary dayside is visible to the observer before or after (a).
    Formula: $\theta=\arcsin(\frac{R_{star}}{a})$"""
    universal = init_const("universal", directory=path_to_constants)

    star_planet_separation_angle = np.rad2deg(
        np.arcsin(star_radius.data / (planet_semi_major_axis.data * universal.au.data))
    )
    return star_planet_separation_angle

We calculate emission spectra as a straight average of the spectra at two phase angles, when a maximum area of the planetary dayside is visible to the observer before and after the secondary eclipse. The star-planet separation angle required for that is found using:

$\sin{\theta}=\frac{R_{star}}{a}$, where $R_{star}$ is the stellar radius [m], and $a$ is the planetary semi-major axis [m], so $\theta=\arcsin(\frac{R_{star}}{a})$.

1. Find the star-planet separation angle

In [6]:
thetas = {}
for planet, planet_const in zip(PLANETS.keys(), PLANET_CONSTS):
    star_planet_separation_angle = calc_star_planet_separation_angle(
        planet_const.star_radius, planet_const.planet_semi_major_axis
    )
    thetas[planet] = {"star_planet_separation_angle": star_planet_separation_angle}
thetas

{'hatp11b': {'star_planet_separation_angle': 3.4616338231995294},
 'hd189733b': {'star_planet_separation_angle': 6.388682667512781},
 'hd209458b': {'star_planet_separation_angle': 6.533274226136118},
 'wasp17b': {'star_planet_separation_angle': 8.240204604688666}}

2. Determine the phase angle increment between two subsequent radiation timesteps by running the UM for 1 radiation timestep (in this work occuring every 5 dynamics timesteps)

In [7]:
timings = {
    "hatp11b": {
        "start_ts": 2880001,  # dynamics timestep
        "start_phase_ang": np.rad2deg(2.27761361148305),
        "d_phase_ang_dt": np.rad2deg(2.27984526201999 - 2.27761361148305) / 5,
    },
    "hd189733b": {
        "start_ts": 2880001,
        "start_phase_ang": np.rad2deg(2.94442707889721),
        "d_phase_ang_dt": np.rad2deg(2.94934294542418 - 2.94442707889721) / 5,
    },
    "hd209458b": {
        "start_ts": 2880001,
        "start_phase_ang": np.rad2deg(3.74472150325983),
        "d_phase_ang_dt": np.rad2deg(3.74781605867111 - 3.74472150325983) / 5,
    },
    "wasp17b": {
        "start_ts": 2880001,
        "start_phase_ang": np.rad2deg(4.44258724628298),
        "d_phase_ang_dt": np.rad2deg(4.44550781062708 - 4.44258724628298) / 5,
    },
}

In [8]:
# Merge dictionaries
vrbls = {k: {**thetas[k], **timings[k]} for k in thetas.keys()}
vrbls

{'hatp11b': {'star_planet_separation_angle': 3.4616338231995294,
  'start_ts': 2880001,
  'start_phase_ang': 130.49764729952798,
  'd_phase_ang_dt': 0.02557283142295282},
 'hd189733b': {'star_planet_separation_angle': 6.388682667512781,
  'start_ts': 2880001,
  'start_phase_ang': 168.70324470484357,
  'd_phase_ang_dt': 0.05633168092900505},
 'hd209458b': {'star_planet_separation_angle': 6.533274226136118,
  'start_ts': 2880001,
  'start_phase_ang': 214.5567375886734,
  'd_phase_ang_dt': 0.03546099290714304},
 'wasp17b': {'star_planet_separation_angle': 8.240204604688666,
  'start_ts': 2880001,
  'start_phase_ang': 254.5414993306612,
  'd_phase_ang_dt': 0.033467202142663896}}

3. Choose planet and phase angles to create UM output for

In [9]:
# Choose planet
planet = "hd209458b"  # hatp11b, hd189733b, hd209458b, wasp17b
start_ts = vrbls[planet]["start_ts"]
start_phase_ang = vrbls[planet]["start_phase_ang"]
d_phase_ang_dt = vrbls[planet]["d_phase_ang_dt"]
theta = vrbls[planet]["star_planet_separation_angle"]
# Choose phase angles
phase_angs = [
    180 - theta,
    180,  # secondary eclipse
    180 + theta,
    210,
    240,
    270,
    300,
    330,
    360,
    390,
    420,
    450,
    480,
    510,
    540,  # secondary eclipse
]

In [10]:
# hatp11b
np.rad2deg(3.14126236931043 - 2.99843673494070)

8.183306055664154

In [11]:
# hd189733b
np.rad2deg(3.14106174000426 - 3.02799680986755)

6.478143307775015

In [12]:
# hd209458b
np.rad2deg(3.14004537588380 - 2.99769582696263)

8.156028368774065

In [13]:
# wasp17b
np.rad2deg(3.14188471002399 - 2.99877705716654)

8.199464524755179

4. Find radiation timesteps closest to the chosen phase angles

In [14]:
# List of radiation timesteps over two orbits
rad_ts = np.arange(start_ts, start_ts + np.ceil(360 * 2 / d_phase_ang_dt), 5, dtype=int)

rad_ts_closest_to_phase_ang = []
for phase_ang in phase_angs:
    # Take the next orbit if the starting UM timestep corresponds to a phase angle > 180
    if start_phase_ang > 180:  # true for hd209458b, wasp17b
        phase_ang += 360
    exact_ts = start_ts + int((phase_ang - start_phase_ang) / d_phase_ang_dt)
    idx = np.argmin(np.abs(rad_ts - exact_ts))
    rad_ts_closest_to_phase_ang.append(rad_ts[idx])

    print(np.deg2rad(phase_ang % 360), phase_ang % 360, rad_ts[idx])

3.027565507408034 173.4667257738639 2888996
3.141592653589793 180 2889176
3.2556197997715524 186.5332742261361 2889361
3.6651914291880923 210 2890026
4.1887902047863905 240 2890871
4.71238898038469 270 2891716
5.235987755982989 300 2892561
5.759586531581287 330 2893406
0.0 0 2894256
0.5235987755982988 30 2895101
1.0471975511965976 60 2895946
1.5707963267948966 90 2896791
2.0943951023931953 120 2897636
2.6179938779914944 150 2898486
3.141592653589793 180 2899331
